# Announcement

As with wrangling, I switched this notebook over to use the Titanic dataset for consistency. The video uses the Pima dataset.

The video ends rather abuptly. I hope the notebook is clear and what you need to do.

One thing I introduce is Upsampling. I promised I would do this way back in Chapter 2. I do not expect you to use it but thought it was worth demonstrating.

I also provide an optional make-up problem for you to get points back.

At the end of the notebook, I explore several ways of combining the four models. The most sophisticated is something called stacking. In essence we build a meta-model that we train with the output of the existing models. So this meta-model attempts to learn how to interpret the existing models' output.

<center>
<h1>Training and Tuning</h1>
</center>

<hr>

Once you are done here, you are ready to start playing with your server. Cool.

# How long does it take?

I think tuning time is the biggest issue for you now.
Using Pima data (training set = 614 rows) and what I consider an ok set of parameters to tune, this notebook takes me roughly 3 hours.

Take away is that as you tune each model, be aware that you might need to leave it running while you do something else.

The good news is that each model-tuning step is independent. Once you tune model X and save to GitHub, you are done with model X and can move on to model Y. The bad news is that if your dataset is larger, e.g., 5K rows, you can expect an increase in my times.

The further bad news is that there is not an easy way to get a progress bar with HalvingSearch. So if you wait 30 minutes, you don't know if you are almost done or will take another 4 hours.

Here are some strategies to consider:

1. Use incremental tuning. Tune some subset of params. Get best values and fix them. Then take on new subset using fixed values from past. You can use this strategy with both halving search and keras tuner.

2. I have factor=3. You could increase it to reduce wait time. But my experimentation tells me you may not gain that much.

3. For keras_tuner, it's easier. You can play around with `max_trials`. Set it small to start, e.g., 5. You can count on linearity here. If 5 trials takes 10 minutes, 50 likely to take 100 minutes.

## Set-up

First bring in your library.

In [30]:
github_name = 'marvnc'
repo_name = 'cs523'
source_file = 'library.py'
url = f'https://raw.githubusercontent.com/{github_name}/{repo_name}/main/{source_file}'
!rm $source_file
!wget $url
%run -i $source_file
# from library import *

--2025-06-06 05:59:45--  https://raw.githubusercontent.com/marvnc/cs523/main/library.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 49866 (49K) [text/plain]
Saving to: ‘library.py’

library.py          100%[===================>]  48.70K  --.-KB/s    in 0.01s   

2025-06-06 05:59:46 (3.74 MB/s) - ‘library.py’ saved [49866/49866]



## You need to change this url to point to your own dataset

And good idea to rename variables using "titanic" to something closer to your dataset.

In [31]:
# url = 'https://raw.githubusercontent.com/fickas/asynch_models/main/datasets/titanic_trimmed.csv'  #trimmed version
url = 'https://raw.githubusercontent.com/MarvNC/cs523/refs/heads/main/s25_final_cc_approvals_reduced.csv'

approvals_trimmed = pd.read_csv(url)
approvals_trimmed.head()


,Gender,Age,Debt,YearsEmployed,PriorDefault,Employed,CreditScore,DriversLicense,Income,Approved
0,1,30.83,0.000,1.25,1,1,1,0,0,1
1,0,58.67,4.460,3.04,1,1,6,0,560,1
2,0,24.50,0.500,1.50,1,0,0,0,824,1
3,1,27.83,1.540,3.75,1,1,5,1,3,1
4,1,20.17,5.625,1.71,1,0,0,0,0,1


In [32]:
len(approvals_trimmed)

690

# Break out into features and labels



In [33]:
target_col = 'Approved'

approvals_features = approvals_trimmed.drop(columns=target_col)
labels = approvals_trimmed[target_col].to_list()

In [34]:
labels.count(1)/len(labels)

0.4449275362318841

## Load pipeline from Wrangling notebook

You will be doing this exact same thing in the server.

In [35]:
import joblib

# model_path = 'MarvNC/cs523/s25_'
full_path = f'https://github.com/MarvNC/cs523/raw/refs/heads/main/s25_final_fully_fitted_pipeline.pkl'
!rm 's25_final_fully_fitted_pipeline.pkl'
!wget $full_path
titanic_transformer = joblib.load("s25_final_fully_fitted_pipeline.pkl")


--2025-06-06 05:59:46--  https://github.com/MarvNC/cs523/raw/refs/heads/main/s25_final_fully_fitted_pipeline.pkl
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/MarvNC/cs523/refs/heads/main/s25_final_fully_fitted_pipeline.pkl [following]
--2025-06-06 05:59:46--  https://raw.githubusercontent.com/MarvNC/cs523/refs/heads/main/s25_final_fully_fitted_pipeline.pkl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1327 (1.3K) [application/octet-stream]
Saving to: ‘s25_final_fully_fitted_pipeline.pkl’

s25_final_fully_fit 100%[===================>]   1.30K  --.-KB/s    in 0s      

2025-06-06 05:59:46 (2

# Step I. Break into numpy datasets

In [36]:
%%capture
# rs = 74 #what you computed in wrangling notebook
rs = approvals_variance_based_split
# label_column = 'Survived'  #change to name of your label column
label_column = target_col

x_train,  x_test, y_train,  y_test = dataset_setup(approvals_trimmed, label_column, titanic_transformer, rs=rs)

In [37]:
len(x_train)

552

# II. Upsampling

In Chapter 2 I removed duplicates, giving us unique rows. However, it did shrink the table down to roughly 1000. I noted I would show you a way to build the table back up. I am going to use a popular method called SMOTE (Synthetic Minority Over-sampling Technique). I'll show you how to use it even though I do not expect you will need it here. But could come in handy later in your career.

Note that I am only applying it to the training data. I'd like to keep the test data pure: augment training, let test data stand.

You can find plenty of tutorials on SMOTE. Briefly, it generates new rows by
using existing rows as starting places and then interpolating values. So it
does not duplicate rows but tries to give you similar rows.

In [38]:
from imblearn.over_sampling import SMOTE

# Calculate target numbers for 3000 total samples
target_total = 2000
pos_count = np.sum(y_train == 1)/len(y_train)
neg_count = np.sum(y_train == 0)/len(y_train)
target_0 = int(neg_count * target_total)  # 1950 samples
target_1 = int(pos_count * target_total)  # 1050 samples

# Create SMOTE instance with specified sampling strategy
smote = SMOTE(sampling_strategy={0: target_0, 1: target_1}, random_state=42)
x_resampled, y_resampled = smote.fit_resample(x_train, y_train)  #requires transformed data - cannot handle categorical columns

# Verify the new distribution
print("New class distribution:")
print(f"Class 0: {sum(y_resampled == 0)} ({sum(y_resampled == 0)/len(y_resampled):.2%})")
print(f"Class 1: {sum(y_resampled == 1)} ({sum(y_resampled == 1)/len(y_resampled):.2%})")
print(f"Total samples: {len(y_resampled)}")

New class distribution:
Class 0: 1108 (55.43%)
Class 1: 891 (44.57%)
Total samples: 1999


In [39]:
# Uncomment if you want to use upsampled data

x_train= x_resampled
y_train = y_resampled

# Prof said to upsample to 2k

# III. Setup Lime

Reminder: Lime will help us explain to the user why we come up with the predictions we do.

In [40]:
%%capture
!pip install lime

In [41]:
import lime
from lime import lime_tabular

In [42]:
feature_names = approvals_features.columns.to_list()
print(feature_names)

['Gender', 'Age', 'Debt', 'YearsEmployed', 'PriorDefault', 'Employed', 'CreditScore', 'DriversLicense', 'Income']


### Set up the explainer before using

In [43]:
explainer = lime.lime_tabular.LimeTabularExplainer(x_train,
                    feature_names=feature_names,
                    training_labels=y_train,
                    class_names=[0,1], #label values
                    verbose=True,
                    mode='classification')



# IV. Write out to file

And move to GitHub.

In [44]:
!pip install dill
import dill as pickle
with open('lime_explainer.pkl', 'wb') as file:
    pickle.dump(explainer, file)

#read it back in just as a test
with open('lime_explainer.pkl', 'rb') as file:   #this will be in your webserver
    explainer2 = pickle.load(file)

# Minimal help from me with remainder of notebook

I can remind you of the steps you need for each model's tuning:

1. If using halving search, set up grid. If using Optuna, then set up model builder with hp code. With Optuna, will also need to define a validation set.

2. Get the best model found by tuning.

3. Run it on test set.

4. Produce threshold table.

5. Save both best model and threshold table out to GitHub so can load them back in with server.

I would avoid Run All here. Each notebook can be tuned separately, really in any order. But once you finish steps above for one model, you don't want to waste time and repeat them.

# V. KNN tuning



In [45]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_auc_score

### Follow the steps

In [46]:
knn_grid_raw = dict(n_neighbors=range(5,50,10),
                weights=('distance', 'uniform'),
                algorithm=('brute', 'kd_tree', 'ball_tree', 'auto'),
                p=(2,1)  #When p=1, manhattan_distance, when p=2 euclidean_distance.
)

knn_grid = sort_grid(knn_grid_raw)
knn_grid

{'algorithm': ['auto', 'ball_tree', 'brute', 'kd_tree'],
 'n_neighbors': [5, 15, 25, 35, 45],
 'p': [1, 2],
 'weights': ['distance', 'uniform']}

In [47]:
from sklearn.model_selection import ParameterGrid

param_grid = ParameterGrid(knn_grid)  #a list of dictionaries, one for each combo
len(param_grid)  #160

80

In [48]:
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV
knn_model = KNeighborsClassifier()

In [49]:
%%capture
knn_grid_result = halving_search(knn_model,knn_grid,x_train,y_train)
knn_best_model = knn_grid_result.best_estimator_

In [50]:
knn_grid_result.best_params_

{'algorithm': 'brute', 'n_neighbors': 35, 'p': 1, 'weights': 'distance'}

In [51]:
knn_best_model.score(x_test, y_test)

0.8115942028985508

## Build Threshold Table

In [52]:
ypos = knn_best_model.predict_proba(x_test)[:,1]  #probability of positive class

In [53]:
result_df, fancy_df = threshold_results(np.round(np.arange(0.0, 1.01, .05), 2), y_test, ypos)
fancy_df

,threshold,precision,recall,f1,auc,accuracy
0,0.00,0.44,1.00,0.61,0.90,0.44
1,0.05,0.57,0.97,0.72,0.90,0.67
2,0.10,0.63,0.95,0.76,0.90,0.73
3,0.15,0.71,0.92,0.80,0.90,0.80
4,0.20,0.77,0.92,0.84,0.90,0.84
5,0.25,0.79,0.87,0.83,0.90,0.84
6,0.30,0.79,0.85,0.82,0.90,0.83
7,0.35,0.82,0.82,0.82,0.90,0.84
8,0.40,0.83,0.82,0.83,0.90,0.85
9,0.45,0.83,0.80,0.82,0.90,0.84


In [54]:
result_df

,threshold,precision,recall,f1,auc,accuracy
0,0.00,0.44,1.00,0.61,0.9,0.44
1,0.05,0.57,0.97,0.72,0.9,0.67
2,0.10,0.63,0.95,0.76,0.9,0.73
3,0.15,0.71,0.92,0.80,0.9,0.80
4,0.20,0.77,0.92,0.84,0.9,0.84
5,0.25,0.79,0.87,0.83,0.9,0.84
6,0.30,0.79,0.85,0.82,0.9,0.83
7,0.35,0.82,0.82,0.82,0.9,0.84
8,0.40,0.83,0.82,0.83,0.9,0.85
9,0.45,0.83,0.80,0.82,0.9,0.84


## Save Results

In [55]:
result_df.to_csv('s25_final_knn_thresholds.csv', index=False)

In [56]:
from joblib import dump
dump(knn_best_model, 's25_final_knn_model.joblib')

['s25_final_knn_model.joblib']

# VI. Logistic Regression tuning



In [57]:
from sklearn.linear_model import LogisticRegressionCV

### Follow the steps

In [58]:
# create grid

logreg_grid_raw = {
    'Cs': [5,10,15],
    'cv': [3,5,10],
    'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
    'max_iter': [10,100,500,1000]
}

In [59]:
logreg_grid = sort_grid(logreg_grid_raw)
logreg_grid

{'Cs': [5, 10, 15],
 'cv': [3, 5, 10],
 'max_iter': [10, 100, 500, 1000],
 'solver': ['lbfgs', 'liblinear', 'newton-cg', 'sag', 'saga']}

In [60]:
param_grid = ParameterGrid(logreg_grid)
len(param_grid)  #180

180

In [61]:
logreg_model = LogisticRegressionCV(random_state=1, n_jobs=-1)  #base model


In [62]:
%%capture
grid_result = halving_search(logreg_model, logreg_grid, x_train, y_train)
best_logreg_model = grid_result.best_estimator_

In [63]:
grid_result.best_params_


{'Cs': 10, 'cv': 3, 'max_iter': 10, 'solver': 'sag'}

In [64]:
ypos = best_logreg_model.predict_proba(x_test)[:,1]

In [65]:
result_df, fancy_df = threshold_results(np.round(np.arange(0.0,1.01,.05), 2), y_test, ypos)
fancy_df

,threshold,precision,recall,f1,auc,accuracy
0,0.00,0.44,1.00,0.61,0.90,0.44
1,0.05,0.44,1.00,0.61,0.90,0.44
2,0.10,0.58,0.93,0.71,0.90,0.67
3,0.15,0.67,0.93,0.78,0.90,0.77
4,0.20,0.74,0.93,0.83,0.90,0.83
5,0.25,0.75,0.93,0.83,0.90,0.83
6,0.30,0.78,0.93,0.85,0.90,0.86
7,0.35,0.78,0.93,0.85,0.90,0.86
8,0.40,0.78,0.92,0.84,0.90,0.85
9,0.45,0.79,0.87,0.83,0.90,0.84


In [66]:
result_df

,threshold,precision,recall,f1,auc,accuracy
0,0.00,0.44,1.00,0.61,0.9,0.44
1,0.05,0.44,1.00,0.61,0.9,0.44
2,0.10,0.58,0.93,0.71,0.9,0.67
3,0.15,0.67,0.93,0.78,0.9,0.77
4,0.20,0.74,0.93,0.83,0.9,0.83
5,0.25,0.75,0.93,0.83,0.9,0.83
6,0.30,0.78,0.93,0.85,0.9,0.86
7,0.35,0.78,0.93,0.85,0.9,0.86
8,0.40,0.78,0.92,0.84,0.9,0.85
9,0.45,0.79,0.87,0.83,0.9,0.84


In [67]:
result_df.to_csv('s25_final_logreg_thresholds.csv', index=False)

In [68]:
dump(best_logreg_model, 's25_final_logreg_model.joblib')

['s25_final_logreg_model.joblib']

# VII. LGB tuning



In [69]:
from lightgbm import LGBMClassifier

### Follow the steps

In [70]:
lgb_grid_raw = {
    # Parameters that work well with HalvingGridSearch
    "n_estimators": [5, 10, 50, 100],
    "learning_rate": [.01, .05, 0.1, 0.2, 0.3, 0.4],
    "max_depth": [1,5,10,15],
    "boosting_type": ['gbdt', 'dart', 'goss'],
    "num_leaves": [5, 7, 15, 31, 40],   #start small with binary classification

    "min_child_samples": [5, 10,15],
    "class_weight": ['balanced', None],

    #"subsample": [0.25, 0.5, 0.75]     # Drop this as HalvingGridSearch handles row sampling
    #"subsample_freq": [0, 1, 5]        # Drop this too as it's related to subsample
}

lgb_grid = sort_grid(lgb_grid_raw)
lgb_grid

{'boosting_type': ['dart', 'gbdt', 'goss'],
 'class_weight': ['balanced', None],
 'learning_rate': [0.01, 0.05, 0.1, 0.2, 0.3, 0.4],
 'max_depth': [1, 5, 10, 15],
 'min_child_samples': [5, 10, 15],
 'n_estimators': [5, 10, 50, 100],
 'num_leaves': [5, 7, 15, 31, 40]}

In [71]:
param_grid = ParameterGrid(lgb_grid)  #a list of dictionaries, one for each combo
len(param_grid)  #8640

8640

In [72]:
import datetime  #can use to time tuning - roughly 25 minutes for below!

In [73]:
%%capture
start = datetime.datetime.now()

grid_result = halving_search(LGBMClassifier(random_state=1234), lgb_grid, x_train, y_train)

end = datetime.datetime.now()
time_difference = end - start
difference_in_minutes = time_difference.total_seconds() / 60

In [74]:
print(f"The difference between the two datetimes is {difference_in_minutes} minutes.")

The difference between the two datetimes is 17.4703484 minutes.


In [75]:
lgb_best_model = grid_result.best_estimator_
grid_result.best_params_

{'boosting_type': 'gbdt',
 'class_weight': 'balanced',
 'learning_rate': 0.1,
 'max_depth': 15,
 'min_child_samples': 15,
 'n_estimators': 100,
 'num_leaves': 40}

In [77]:
result_df, fancy_df = threshold_results(np.round(np.arange(0.0,1.01,.05), 2), y_test, lgb_best_model.predict_proba(x_test)[:,1])
fancy_df

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


,threshold,precision,recall,f1,auc,accuracy
0,0.00,0.44,1.00,0.61,0.93,0.44
1,0.05,0.77,0.97,0.86,0.93,0.86
2,0.10,0.76,0.90,0.83,0.93,0.83
3,0.15,0.77,0.89,0.82,0.93,0.83
4,0.20,0.79,0.87,0.83,0.93,0.84
5,0.25,0.79,0.87,0.83,0.93,0.84
6,0.30,0.79,0.87,0.83,0.93,0.84
7,0.35,0.79,0.87,0.83,0.93,0.84
8,0.40,0.79,0.87,0.83,0.93,0.84
9,0.45,0.80,0.85,0.83,0.93,0.84


In [78]:
result_df

,threshold,precision,recall,f1,auc,accuracy
0,0.00,0.44,1.00,0.61,0.93,0.44
1,0.05,0.77,0.97,0.86,0.93,0.86
2,0.10,0.76,0.90,0.83,0.93,0.83
3,0.15,0.77,0.89,0.82,0.93,0.83
4,0.20,0.79,0.87,0.83,0.93,0.84
5,0.25,0.79,0.87,0.83,0.93,0.84
6,0.30,0.79,0.87,0.83,0.93,0.84
7,0.35,0.79,0.87,0.83,0.93,0.84
8,0.40,0.79,0.87,0.83,0.93,0.84
9,0.45,0.80,0.85,0.83,0.93,0.84


In [80]:
result_df.to_csv('s25_final_lgb_thresholds.csv', index=False)
dump(lgb_best_model, 's25_final_lgb_model.joblib')

['s25_final_lgb_model.joblib']

# VIII. ANN tuning



In [81]:
!pip install keras-tuner -q
import keras_tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.2 MB/s eta 0:00:00


In [82]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Input
import tensorflow as tf
from tensorflow import keras

In [83]:
tf.keras.utils.set_random_seed(1234)  #need this for replication
tf.config.experimental.enable_op_determinism()  #ditto - https://www.tensorflow.org/api_docs/python/tf/config/experimental/enable_op_determinism
tf.config.threading.set_inter_op_parallelism_threads(1)
tf.config.threading.set_intra_op_parallelism_threads(1)

In [84]:
import hashlib

def string_to_seed(string):
    # Create a hash of the string using SHA-256
    hash_object = hashlib.sha256(string.encode())
    # Convert first 8 bytes of hash to integer
    hash_int = int.from_bytes(hash_object.digest()[:8], 'big')
    return hash_int % (2**32 - 1)

In [85]:
early_stop_cb = tf.keras.callbacks.EarlyStopping(
    monitor='loss',
    min_delta=0,
    patience=10,
    verbose=0
)

### Follow the steps

In [86]:
class MyHyperModel(keras_tuner.HyperModel):
    def __init__(self, n=len(feature_names), metrics='auc',
                  layers=(1,5,1),   # min, max, step
                  units=(4,16,1),    # min, max, step
                  afn_list=('relu', 'leaky_relu')):
        self.n = n # number of features
        self.metrics = metrics
        self.units = units
        self.layers = layers
        self.afn_list = afn_list

    def build(self, hp):
        n = self.n
        metrics = self.metrics
        min_units = self.units[0]
        max_units = self.units[1]
        step_units = self.units[2]
        min_layers = self.layers[0]
        max_layers = self.layers[1]
        step_layers = self.layers[2]
        n_afns = len(self.afn_list)

        ann_model = Sequential()

        # Input layer
        ann_model.add(Input(shape=(n,), name="input_layer"))

        # Hidden layers
        layers = hp.Int("layers", min_value=min_layers, max_value=max_layers, step=step_layers)
        for i in range(layers):
            layer_name = f"hidden_layer_{i}"
            ann_model.add(Dense(
                      name=layer_name+'_dense',
                      kernel_regularizer=tf.keras.regularizers.l2(0.01),
                      kernel_initializer=tf.keras.initializers.HeNormal(seed=string_to_seed(layer_name+'_dense')),
                      units=hp.Int(f"hidden_units{i}", min_value=min_units, max_value=max_units, step=step_units),
                      activation=self.afn_list[hp.Int(f'afn{i}', min_value=0, max_value=n_afns-1, step=1)]
            ))

        # Output layer
        ann_model.add(Dense(units=1, activation='sigmoid'))

        ann_model.compile(loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=.1),
                      optimizer=tf.keras.optimizers.Adam(learning_rate=.001),
                      metrics=['auc', 'accuracy'])
        return ann_model


In [87]:
val_size = 250
x_train_tune = x_train[:-val_size]
x_train_val = x_train[-val_size:]
y_train_tune = y_train[:-val_size]
y_train_val = y_train[-val_size:]
max_trials = 40

In [88]:
hyper_tuner = keras_tuner.RandomSearch(
    hypermodel=MyHyperModel(),
    objective=keras_tuner.Objective('val_auc', 'max'),
    max_trials=max_trials,
    executions_per_trial=1,
    overwrite=True,
    seed=1234,
)

In [89]:
import datetime
start = datetime.datetime.now()

callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor='val_auc',
        mode='max',
        patience=5,
        restore_best_weights=True)
]

hyper_tuner.search(x_train_tune, y_train_tune,
                  epochs=100, batch_size=32,
                  validation_data=(x_train_val, y_train_val),
                  callbacks=callbacks)

tf.keras.backend.clear_session()  # Clear unused models

end = datetime.datetime.now()
time_difference = end - start
difference_in_minutes = time_difference.total_seconds() / 60
print(f"Tuning took {difference_in_minutes:.1f} minutes")


Trial 40 Complete [00h 00m 06s]
val_auc: 0.0

Best val_auc So Far: 0.0
Total elapsed time: 00h 04m 31s
Tuning took 4.5 minutes


In [90]:
best_hp = hyper_tuner.get_best_hyperparameters()[0]
print(best_hp.values)

{'layers': 5, 'hidden_units0': 15, 'afn0': 1, 'hidden_units1': 4, 'afn1': 0, 'hidden_units2': 4, 'afn2': 0, 'hidden_units3': 4, 'afn3': 0, 'hidden_units4': 4, 'afn4': 0}


In [91]:
hypermodel = MyHyperModel()
final_ann_model = hypermodel.build(best_hp)
training = final_ann_model.fit(x_train, y_train,
                            epochs=100,
                            validation_data=(x_train_val, y_train_val),
                            callbacks=[early_stop_cb])

Epoch 1/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.5348 - auc: 0.5653 - loss: 1.2934 - val_accuracy: 0.0840 - val_auc: 0.0000e+00 - val_loss: 1.2212
Epoch 2/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5714 - auc: 0.6036 - loss: 1.1863 - val_accuracy: 0.0040 - val_auc: 0.0000e+00 - val_loss: 1.1413
Epoch 3/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.5500 - auc: 0.6865 - loss: 1.0991 - val_accuracy: 0.0000e+00 - val_auc: 0.0000e+00 - val_loss: 1.0753
Epoch 4/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.5475 - auc: 0.7555 - loss: 1.0275 - val_accuracy: 0.0000e+00 - val_auc: 0.0000e+00 - val_loss: 1.0195
Epoch 5/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.5565 - auc: 0.8093 - loss: 0.9679 - val_accuracy: 0.1640 - val_auc: 0.0000e+00 - val_loss: 0.9645
Epoch 6/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.6714 - auc: 0.8516 - loss: 0.9139 - val_accuracy: 0.7720 - val_auc: 0.0000e+00 - val_loss: 0.8302
Epoch 7/100
6

In [92]:
ypos = final_ann_model.predict(x_test)[:,0]  # Probability of positive class
result_df, fancy_df = threshold_results(np.round(np.arange(0.0, 1.01, .05), 2), y_test, ypos)
fancy_df

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step


,threshold,precision,recall,f1,auc,accuracy
0,0.00,0.44,1.00,0.61,0.92,0.44
1,0.05,0.44,1.00,0.61,0.92,0.44
2,0.10,0.44,1.00,0.61,0.92,0.44
3,0.15,0.77,0.93,0.84,0.92,0.85
4,0.20,0.78,0.93,0.85,0.92,0.86
5,0.25,0.78,0.93,0.85,0.92,0.86
6,0.30,0.78,0.93,0.85,0.92,0.86
7,0.35,0.79,0.93,0.86,0.92,0.86
8,0.40,0.79,0.93,0.86,0.92,0.86
9,0.45,0.79,0.89,0.84,0.92,0.85


In [93]:
result_df.to_csv('s25_final_ann_thresholds.csv', index=False)
from tensorflow import keras
final_ann_model.save('s25_final_ann_model.keras')

# You should eventually have these files on GitHub

* LIME explainer
* tuned KNN model and associated threshold table
* tuned logistic regression model and associated threshold table
* tuned light boosting model and associated threshold table
* tuned ANN model and associated threshold table

# Optional make-up: Random Forest model

I will give you credit for one homework assignment in terms of points if you elect to take on this problem.

You will need to do two things: (1) tune and save your threshold table and model below, and (2) add the model to your production notebook (your last notebook that is part of final.) The latter is the most tricky given you will actually have to change several places in the code I handed you for server. But it is doable if you get an early jump on it.

In [ ]:
# From chapter 12
from sklearn.ensemble import RandomForestClassifier

### Follow the steps

## You still need to change the production notebook

Find the places where you are loading models and thresholds and add the RF results. Find places where you are doing predictions and add RF prediction. Find place where you are showing prediction results in html and add RF prediction. Also add threshold table.

This should not take long but will require you to pay attention to what you are doing to avoid screwing up what is already there.

# Just for your interest

There are several ways to combine the results of multiple models, four models in our case. We are using one of the ways in the server, but wanted to show you other options.

# IX. Voting - averaging binary

There are two ways I can see of voting when have 4 models producing results. The first is to convert their output to binary. Then simply look for majority of either 0s or 1s. I added a twist that I fall back on probabilities for ties.

In [95]:
lgb_raw = lgb_best_model.predict_proba(x_test)[:,1]
knn_raw = knn_best_model.predict_proba(x_test)[:,1]
logreg_raw = best_logreg_model.predict_proba(x_test)[:,1]
ann_raw = final_ann_model.predict(x_test)[:,0]

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


In [96]:
yvotes = []
for i in range(len(y_test)):
  the_vote = (lgb_raw[i]>=.5+logreg_raw[i]>=.5+knn_raw[i]>=.5+ann_raw[i]>=.5)
  if the_vote==2:
    #tie breaker - go to probabilities
    prob = (knn_yraw[i]+logreg_yraw[i]+xgb_yraw[i]+ann_yraw[i])/4
    the_winner = 1 if prob>=.5 else 0
  else:
    the_winner = 1 if the_vote>2 else 0
  yvotes.append(the_winner)

In [97]:
sum([1 if p>=.5 else 0 for p in ann_raw])/len(x_test)

0.47101449275362317

In [98]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, yvotes)
print(cm)


[[77  0]
 [61  0]]


In [99]:
(cm[0,0]+cm[1,1])/len(y_test)  #accuracy 0.5665399239543726

np.float64(0.5579710144927537)

Can now use it to compute precision and recall.

In [100]:
def precision_recall(y_test, y_pred):
    cm = confusion_matrix(y_test, y_pred)
    tp = cm[0,0]
    fp = cm[0,1]
    fn = cm[1,0]
    prec = tp / (tp+fp)
    rec = tp / (tp+fn)
    return prec, rec

precision, recall = precision_recall(y_test, yvotes)
print(f'Precision: {precision} Recall {recall}')

Precision: 1.0 Recall 0.5579710144927537


In [101]:
f1 = 2*(precision*recall)/(precision+recall)
f1

np.float64(0.7162790697674419)

# X. Prob averaging

The second voting approach is not actually voting. Instead, take average of 4 raw probabilities and use result as final probability. Can then run that through threshold table.

This is what the server is doing to get the "Ensemble" value.

In [102]:
avg_yraw = []
for i in range(len(y_test)):
  prob = (knn_raw[i]+logreg_raw[i]+lgb_raw[i]+ann_raw[i])/4
  avg_yraw.append(prob)

In [103]:
result_df, fancy_df = threshold_results(np.linspace(0,1,19,endpoint=True), y_test, avg_yraw)

# XI. Stacking

This is interesting in that it builds a whole separate model (a meta model) that takes the output of other base models, three in example below, and uses that as a row. So a row of 3 feature values, one from each of the base models.

I kind of like it. The meta model learns how to combine the outputs of base models, e.g., when to weight KNN higher than LGB, etc.


In [104]:
from sklearn.ensemble import StackingClassifier

estimators = [
     ('knn', KNeighborsClassifier(15, algorithm='ball_tree', p=1, weights='distance')),
    ('logreg', LogisticRegressionCV(Cs= 5, class_weight= None, cv= 5, max_iter= 500, solver= 'saga', penalty='l1', random_state=1234)),
    ('lgb', LGBMClassifier(boosting_type= 'gbdt',
                          class_weight= 'balanced',
                          learning_rate= 0.3,
                          max_depth= 5,
                          min_child_samples= 10,
                          n_estimators= 10,
                          num_leaves= 7,
                          random_state=1234),
    )
]
final_estimator = LogisticRegressionCV(random_state=1234)   #this is choice for meta model
clf = StackingClassifier(estimators=estimators, final_estimator=final_estimator)

In [105]:
%%capture
clf.fit(x_train, y_train)

In [106]:
yraw = clf.predict_proba(x_test)[:,1]
result_df, fancy_df = threshold_results(np.linspace(0,1,19,endpoint=True), y_test, yraw)
fancy_df

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but LogisticRegressionCV was fitted without feature names
  warnings.warn(


,threshold,precision,recall,f1,auc,accuracy
0,0.00,0.44,1.00,0.61,0.86,0.44
1,0.06,0.63,0.85,0.73,0.86,0.72
2,0.11,0.70,0.84,0.76,0.86,0.77
3,0.17,0.72,0.82,0.77,0.86,0.78
4,0.22,0.75,0.79,0.77,0.86,0.79
5,0.28,0.76,0.79,0.77,0.86,0.80
6,0.33,0.76,0.79,0.77,0.86,0.80
7,0.39,0.75,0.75,0.75,0.86,0.78
8,0.44,0.79,0.75,0.77,0.86,0.80
9,0.50,0.79,0.75,0.77,0.86,0.80


<img src='https://www.dropbox.com/scl/fi/zilmy2diy1lg1tva9vurx/Screenshot-2025-02-07-at-8.38.53-AM.png?rlkey=006szbv5t0daha005eotxt9k2&raw=1' height=400>